In [4]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np
import datetime

Update list of polymers based on raw data collected

In [5]:
polymers = pd.read_excel('Data/Melt Viscosity Data.xlsx', ["Homopolymer", "Copolymer", "Polymer Blend"])
df_master = pd.read_pickle("database_2021_11_30.pkl")
columns = ['rid', 'smiles', 'fingerprints']
df_master = df_master[columns]

In [6]:
SMILES_df = pd.DataFrame(columns = ['Polymer', 'SMILES', 'PID'])
SMILES_df = pd.concat([SMILES_df, polymers["Homopolymer"][['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Copolymer"].rename(columns = {'Polymer 1': 'Polymer', 'SMILES 1': 'SMILES', 'PID 1':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Copolymer"].rename(columns = {'Polymer 2': 'Polymer', 'SMILES 2': 'SMILES', 'PID 2':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Polymer Blend"].rename(columns = {'Polymer 1': 'Polymer', 'SMILES 1': 'SMILES', 'PID 1':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Polymer Blend"].rename(columns = {'Polymer 2': 'Polymer', 'SMILES 2': 'SMILES', 'PID 2':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = SMILES_df.drop_duplicates('Polymer').reset_index().drop(columns=['index'])
SMILES_df['SMILES'] = SMILES_df['SMILES'].astype(str)

In [7]:
SMILES_df.to_excel('Polymer-SMILES.xlsx', sheet_name ='Polymers', index = False)

Update/read in custom fp file

In [57]:
new_fp = pd.read_csv('SMILES-2022-01-10-FP.csv')
custom_fp = pd.read_excel('Data/custom_fp.xlsx')
custom_fp.append(new_fp).to_excel('Data/custom_fp.xlsx', index = False)

Read in updated list with SMILES to get polymer fingerprints

In [8]:
custom_fp = pd.read_excel('Data/custom_fp.xlsx')
file_name = 'Polymer-SMILES.xlsx'
SMILES_df = pd.read_excel(file_name)
IDcon = pd.read_excel('Data/polymer_master_data_v20200625_1.xlsx')
SMILES_df['SMILES'] = SMILES_df['SMILES'].astype(str)

In [9]:
df_master

,rid,smiles,fingerprints
0,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
1,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
2,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
3,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
4,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
...,...,...,...
29807,R1014114,S=C([*])CCCC(C(C)(C)C)S[*],NaN
29808,R1014115,S=C([*])CCCC(C(C)C(C)C)S[*],NaN
29809,R1014116,S=C([*])CCCC(CC(C)(C)C)S[*],NaN
29810,R1014117,S=C([*])CCCC(C(C)C(C)(C)C)S[*],NaN


In [10]:
cust_fp_col = [col for col in custom_fp.columns if 'fp' in col]
fp_df = pd.DataFrame()
need_fp = pd.DataFrame()
need_fp.index.names = ['INDEX']
for i in SMILES_df.index:
    if '*' in SMILES_df.loc[i, 'SMILES'] or 'C' in SMILES_df.loc[i, 'SMILES']:
        if len(df_master.loc[df_master['smiles'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True)) == 0:
            if len(custom_fp.loc[custom_fp['SMILES'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True)) == 0:
                print('Need FP for ' + SMILES_df.loc[i, 'SMILES'])
                need_fp.loc[len(need_fp), 'SMILES'] = SMILES_df.loc[i, 'SMILES']
                continue
            else:
                fp = pd.DataFrame(custom_fp.loc[custom_fp['SMILES'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True).loc[0, cust_fp_col])
                print(fp)
        else:            
            fp = pd.DataFrame(df_master.loc[df_master['smiles'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True).loc[0, 'fingerprints'], index = [i])
        fp.loc[i, 'SMILES'] = SMILES_df.loc[i, 'SMILES']
        fp.loc[i, 'Polymer'] = SMILES_df.loc[i, 'Polymer']
        fp_df = fp_df.append(fp, ignore_index = True)
    elif pd.isna(SMILES_df.loc[i, 'PID']) == False:
        if len( IDcon.loc[IDcon['PID'] == SMILES_df.loc[i, 'PID'], 'RID'].values) == 0:
            print('Need SMILES for ' + str(SMILES_df.loc[i, 'PID']))
            continue
        RID = IDcon.loc[IDcon['PID'] == SMILES_df.loc[i, 'PID'], 'RID'].values[0]
        fp = pd.DataFrame(df_master.loc[df_master['rid'] == RID].reset_index(drop = True).loc[0, 'fingerprints'], index = [i])
        fp.loc[i, 'SMILES'] = df_master.loc[df_master['rid'] == RID].reset_index(drop = True).loc[0, 'smiles']
        fp.loc[i, 'Polymer'] = SMILES_df.loc[i, 'Polymer']
        fp_df = fp_df.append(fp, ignore_index = True)

                     0
afp_H1_C4_H1  0.290323
afp_C4_C4_H1  1.032258
afp_H1_C4_O2  0.129032
afp_C4_C4_C4  0.225806
afp_C4_C4_O2  0.064516
...                ...
bfp_295            0.0
bfp_314            0.0
bfp_343            0.0
bfp_375            0.0
bfp_464            0.0

[124 rows x 1 columns]
                0
afp_H1_C4_H1  0.0
afp_C4_C4_H1  0.0
afp_H1_C4_O2  0.0
afp_C4_C4_C4  0.0
afp_C4_C4_O2  0.0
...           ...
bfp_295       0.0
bfp_314       0.0
bfp_343       0.0
bfp_375       0.0
bfp_464       0.0

[124 rows x 1 columns]
                     0
afp_H1_C4_H1  0.333333
afp_C4_C4_H1  0.833333
afp_H1_C4_O2  0.055556
afp_C4_C4_C4  0.166667
afp_C4_C4_O2  0.111111
...                ...
bfp_295            0.0
bfp_314            0.0
bfp_343            0.0
bfp_375            0.0
bfp_464            0.0

[124 rows x 1 columns]
                     0
afp_H1_C4_H1   0.27027
afp_C4_C4_H1  0.567568
afp_H1_C4_O2       0.0
afp_C4_C4_C4  0.243243
afp_C4_C4_O2       0.0
...                ...

In [11]:
cols = list(fp_df)
cols.insert(0, cols.pop(cols.index('SMILES')))
cols.insert(0, cols.pop(cols.index('Polymer')))
fp_df = fp_df[cols].drop_duplicates('SMILES').fillna(0)

In [12]:
fp_df

,Polymer,SMILES,afp_C4_C4_C4,afp_C4_C4_H1,afp_H1_C4_H1,bfp_267,bfp_282,bfp_289,bfp_304,bfp_315,...,mfp_MQNs34,bfp_244,bfp_290,bfp_456,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210,bfp_482
0,poly(prop-1-ene),[*]CC([*])C,0.444444,1.111111,0.444444,0.444444,0.444444,0.333333,1.333333,0.222222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,polyisobutylene,[*]CC([*])(C)C,0.583333,0.833333,0.583333,0.666667,0.000000,0.250000,1.750000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,Linear HDPE,[*]C[*],0.333333,1.333333,0.333333,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,polystyrene,[*]CC([*])c1ccccc1,0.125000,0.375000,0.062500,0.000000,0.250000,0.187500,0.750000,0.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,poly(pent-1-ene),[*]CC([*])CCC,0.400000,1.200000,0.400000,0.266667,0.266667,0.600000,1.200000,0.133333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,"poly{[1-(1,3-dicarboxypropyl)-3,5-dimethyl-2,6...",[*][C@@]1(C(=O)N(C(=O)[C@](C1)(C[*])C)[C@@H](C...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
712,poly(sodium methacrylate),[*][C@@](C[*])(C(=O)O[Na])C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
837,"poly(oxybutane-1,4-diyloxy{5-[(sodiooxy)sulfon...",[*]OC(=O)c1cc(cc(c1)S(=O)(=O)O[Na])C(=O)OCCCC[*],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
841,PPTA,[*]Nc1ccc(NC(=O)c2ccc(C([*])=O)cc2)cc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,1.0,1.0,0.928571,0.928571,0.285714,0.0


In [13]:
if file_name == 'Data/EntanglementMW.xlsx':
    ent_fp = fp_df.join(SMILES_df['Me'])
    ent_fp.to_excel('Data/EntanglementMW_fp.xlsx', sheet_name ='Polymers')

In [14]:
fp_df.to_excel('Polymer-SMILES-FP.xlsx', sheet_name ='Polymers')

In [15]:
need_fp

,SMILES
INDEX,
0,*[C@@H](C*)CCCCC=C


In [52]:
need_fp.to_excel('SMILES-' + str(datetime.date.today()) + '.xlsx')